In [3]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import asdict, is_dataclass, dataclass
from pandas.api.types import is_datetime64_any_dtype
import scipy
import numpy as np
import polars as pl
from typing import Dict, List
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import accuracy_score, log_loss
import bisect

from src.common.AssetData import AssetData
from src.common.AssetDataPolars import AssetDataPolars
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut 
from src.databaseService.OutsourceLoader import OutsourceLoader

from src.simulation.SimulatePortfolio import SimulatePortfolio
from src.strategy.StratBuyAndHold import StratBuyAndHold
from src.simulation.ResultAnalyzer import ResultAnalyzer
from src.common.AssetFileInOut import AssetFileInOut
from src.predictionModule.ConditionalML import ConditionalML
from src.common.DataFrameTimeOperations import DataFrameTimeOperationsPandas as DFTO
from src.predictionModule.CollectionModels import CollectionModels
from src.common.AssetFileInOut import AssetFileInOut

from src.common.AssetDataPolars import AssetDataPolars
from src.common.AssetDataService import AssetDataService
from src.predictionModule.ModelAnalyzer import ModelAnalyzer

from src.featureAlchemy.FeatureMain import FeatureMain
from src.common.DataFrameTimeOperations import DataFrameTimeOperationsPolars as DPl


c:\Users\kimer\Desktop\RandomOdyssey\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
assets=AssetFileInOut("../src/stockGroups/bin").loadDictFromFile("group_debug")

# Convert to Polars for speedup
assetspl: Dict[str, AssetDataPolars] = {}
for ticker, asset in assets.items():
    assetspl[ticker]= AssetDataService.to_polars(asset)

In [6]:
conditionalML = ConditionalML(assetspl)

In [7]:
loadup_name = "CondtionalML_debug_onSubset"
conditionalML.load_data('../src/predictionModule/bin', loadup_name)

conditionalML.loadLGBMModel('../src/predictionModule/bin', loadup_name+"_lgbmModel")

print(conditionalML.metadata)

Data and metadata loaded from ../src/predictionModule/bin\CondtionalML_debug_onSubset.pkl
Model loaded from ../src/predictionModule/bin\CondtionalML_debug_onSubset_lgbmModel.pkl
{'Condtional_params': {'idxLengthOneMonth': 21, 'fouriercutoff': 5, 'multFactor': 6, 'daysAfterPrediction': 21, 'monthsHorizon': 6, 'timesteps': 5, 'classificationInterval': [0, 0.05], 'averageOverDays': 5, 'optuna_trials': 20, 'LGBM_max_depth': 50}, 'LGBMModel_params': {'verbosity': -1, 'n_jobs': -1, 'metric': 'multi_logloss', 'boosting_type': 'gbdt', 'early_stopping_rounds': 10, 'n_estimators': 100, 'lambda_l1': 0.01401000733332533, 'lambda_l2': 0.2177783655314511, 'feature_fraction': 0.057712869650885866, 'num_leaves': 1236, 'max_depth': 50, 'learning_rate': 0.0169939525159139}, 'LGBMModel_accuracy_val': 0.44155844155844154, 'LGBMModel_log_loss_val': 1.0124456977314533, 'LGBMModel_accuracy_test': 0.45054945054945056, 'LGBMModel_log_loss_test': 0.9853458747577701}


In [10]:
X_train = pd.DataFrame(conditionalML.X_train, columns=conditionalML.featureColumnNames)

X_train.shape

conditionalML.featureColumnNames.index("Fourier_ReturnLog_RSME")

117